Things i need to do:
- data vis
- make note of hyperparameter tuning process (most important part of presentation)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score,train_test_split, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier
import itertools as it

In [78]:
data_full = pd.read_csv('movies_classification.csv', index_col='Unnamed: 0')

In [83]:
#dropping obs w no runtime
data = data_full.drop(data_full[data_full.runtime.isnull()].index)

In [105]:
data_full.release_date

0         815011200000000000
1         818985600000000000
2         819590400000000000
3         819590400000000000
4         792374400000000000
                ...         
45461   -9223372036854775808
45462    1321488000000000000
45463    1059696000000000000
45464   -1647216000000000000
45465    1496966400000000000
Name: release_date, Length: 45014, dtype: int64

In [104]:
ind = np.where(data_full['runtime'].isna())[0]

data_full.iloc[ind, :]

,belongs_to_collection,budget,popularity,release_date,revenue,runtime,title,vote_average,vote_count,Animation,...,History,Science Fiction,Mystery,War,Foreign,Music,Documentary,Western,TV Movie,english
634,0,0,0.066123,827366400000000000,0.0,NaN,Peanuts – Die Bank zahlt alles,4.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
635,0,0,0.002229,826761600000000000,1.0,NaN,Happy Weekend,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
644,0,0,0.439989,825552000000000000,0.0,NaN,Und keiner weint mir nach,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
802,0,0,0.106345,835228800000000000,0.0,NaN,Diebinnen,4.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
863,0,0,0.437895,592272000000000000,0.0,NaN,Baton Rouge,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45246,0,0,0.002513,1101945600000000000,0.0,NaN,Villa Henriette,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
45310,0,0,0.030803,1406851200000000000,0.0,NaN,Między nami dobrze jest,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
45313,0,0,0.000102,625449600000000000,0.0,NaN,Ostatni dzwonek,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
45377,0,0,0.006352,112147200000000000,0.0,NaN,Simbad e il califfo di Bagdad,0.0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
y = data.revenue
X = data.drop(['revenue', 'title'], axis = 1)

In [70]:
# splitting test/train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 45)

Not scaling the data: Decision trees and ensemble methods do not require feature scaling to be performed as they are not sensitive to the the variance in the data.

In [71]:
# making base model to find the range of hyperparameters
model = DecisionTreeClassifier(random_state = 1)
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values